In [1]:
from astropy.coordinates import SkyCoord
import numpy as np
from IntegralQuery import SearchQuery, IntegralQuery, Filter, Range
from IntegralPointingClustering import ClusteredQuery
import astropy.io.fits as fits
from astropy.table import Table
from datetime import datetime
import matplotlib.pyplot as plt
import math
from numba import njit
from pyspi.utils.function_utils import find_response_version
from pyspi.utils.response.spi_response_data import ResponseDataRMF
from pyspi.utils.response.spi_response import ResponseRMFGenerator
from pyspi.utils.response.spi_drm import SPIDRM
from pyspi.utils.livedets import get_live_dets
from astromodels import Powerlaw, Log_uniform_prior, Uniform_prior, PointSource, SpectralComponent
from chainconsumer import ChainConsumer
import pymultinest
import os
import astropy.time as at
from scipy.stats import poisson
import pickle

[WARNING ] The naima package is not available. Models that depend on it will not be available
[WARNING ] The GSL library or the pygsl wrapper cannot be loaded. Models that depend on it will not be available.
[WARNING ] The ebltable package is not available. Models that depend on it will not be available
[INFO    ] Starting 3ML!
[WARNING ] no display variable set. using backend for graphics without display (agg)
[WARNING ] ROOT minimizer not available
[WARNING ] PyGMO is not available
[WARNING ] The cthreeML package is not installed. You will not be able to use plugins which require the C/C++ interface (currently HAWC)
[WARNING ] Could not import plugin HAWCLike.py. Do you have the relative instrument software installed and configured?
[WARNING ] Could not import plugin FermiLATLike.py. Do you have the relative instrument software installed and configured?
[WARNING ] No fermitools installed
[WARNING ] Env. variable OMP_NUM_THREADS is not set. Please set it to 1 for optimal performances 

In [2]:
def extract_date_range(path):
    with fits.open(f"{path}/pointing.fits") as file:
        t = Table.read(file[1])
        t1 = at.Time(f'{t["TSTART"][0]+2451544.5}', format='jd')
        t1.format = "isot"
        t2 = at.Time(f'{t["TSTOP"][-1]+2451544.5}', format='jd')
        t2.format = "isot"
    return t1.value, t2.value

def get_scw_ids(path, radius_around_crab, min_time_elapsed, print_results=False):
    p = SkyCoord(83.6333, 22.0144, frame="icrs", unit="deg")
    searchquerry = SearchQuery(position=p, radius=f"{radius_around_crab} degree")
    cat = IntegralQuery(searchquerry)

    f = Filter(SCW_TYPE="POINTING",
               TIME=Range(*extract_date_range(path)))
    scw_ids_all = cat.apply_filter(f, return_coordinates=True, remove_duplicates=True)
    
    scw_ids = []
    
    multiple_files = []
    no_files = []
    no_pyspi = []
    
    num_dets = 19
    eb = np.geomspace(18, 2000, 5)
    emod = np.geomspace(18, 2000, 5)
    for scw_id in scw_ids_all:
        good = True
        with fits.open(f"{path}/pointing.fits") as file:
            t = Table.read(file[1])
            index = np.argwhere(t["PTID_ISOC"]==scw_id[0][:8])
            
            if len(index) < 1:
                no_files.append(scw_id)
                good = False
                continue
                
            elif len(index) > 1:
                multiple_files.append(scw_id)
                good = False
                            
            pointing_info = t[index[-1][0]]
        
            t1 = at.Time(f'{pointing_info["TSTART"]+2451544.5}', format='jd').datetime
            time_start = datetime.strftime(t1,'%y%m%d %H%M%S')
                            
            with fits.open(f"{path}/dead_time.fits") as file2:
                t2 = Table.read(file2[1])
                
                time_elapsed = np.zeros(num_dets)
                
                for i in range(num_dets):
                    for j in index:
                        time_elapsed[i] += t2["LIVETIME"][j[0]*85 + i]
                            
            dets = get_live_dets(time=time_start, event_types=["single"])
                            
            if not np.amin(time_elapsed[dets]) > min_time_elapsed:
                good = False
        
        try: # investigate why this is necessary
            version1 = find_response_version(time_start)
            rsp_base = ResponseDataRMF.from_version(version1)
            rsp1 = ResponseRMFGenerator.from_time(time_start, dets[0], eb, emod, rsp_base)
        except:
            no_pyspi.append(scw_id)
            good = False
            
        if good:
            scw_ids.append(scw_id)
            
    if print_results:
        print("Multiple Files:")
        print(multiple_files)
        print("No Files:")
        print(no_files)
        print("No PySpi:")
        print(no_pyspi)
        print("Good:")
        print(scw_ids)
    
    return np.array(scw_ids)

def create_pair_clusters_crab(
    paths,
    min_angle_dif,
    max_angle_dif,
    max_time_dif,
    radius_around_crab,
    min_time_elapsed
): # check if dets is same in pair?
    output = []
    for path in paths:
        scw_ids = get_scw_ids(path, radius_around_crab, min_time_elapsed)
        cq = ClusteredQuery(scw_ids,
                            angle_weight=0.,
                            time_weight=1./max_time_dif,
                            max_distance=1.,
                            min_ang_distance=min_angle_dif,
                            max_ang_distance=max_angle_dif,
                            cluster_size_range = (2,2),
                            failed_improvements_max = 3,
                            suboptimal_cluster_size = 1,
                            close_suboptimal_cluster_size = 1
                            ).get_clustered_scw_ids()
        for pair in cq[2]:
            output.append((path, pair[0], pair[1]))
    return tuple(output)

In [3]:
def save_pair_clusters(pointings, folder):
    if not os.path.exists(f"./{folder}"):
        os.mkdir(folder)
    with open(f"./{folder}/pointings", "wb") as f:
        pickle.dump(pointings, f)
        
def load_pair_clusters(folder):
    with open(f"./{folder}/pointings", "rb") as f:
        pointings = pickle.load(f)
    return pointings

In [4]:
def extract_pointing_info(path, p_id):
    num_dets = 19
    with fits.open(f"{path}/pointing.fits") as file:
        t = Table.read(file[1])
        index = np.argwhere(t["PTID_ISOC"]==p_id[:8])
        
        if len(index) < 1:
            raise Exception(f"{p_id} not found")

        pointing_info = t[index[-1][0]]
        
        t1 = at.Time(f'{pointing_info["TSTART"]+2451544.5}', format='jd').datetime
        time_start = datetime.strftime(t1,'%y%m%d %H%M%S')
            
    with fits.open(f"{path}/dead_time.fits") as file:
        t = Table.read(file[1])
        
        time_elapsed = np.zeros(num_dets)
        
        for i in range(num_dets):
            for j in index:
                time_elapsed[i] += t["LIVETIME"][j[0]*85 + i]
        
    with fits.open(f"{path}/energy_boundaries.fits") as file:
        t = Table.read(file[1])
        
        energy_bins = np.append(t["E_MIN"], t["E_MAX"][-1])
    
    with fits.open(f"{path}/evts_det_spec.fits") as file:
        t = Table.read(file[1])
        
        counts = np.zeros((num_dets, len(energy_bins)-1))
        for i in range(num_dets):
            for j in index:
                counts[i, : ] += t["COUNTS"][j[0]*85 + i]

    return time_start, time_elapsed, energy_bins, counts

In [5]:
def prepare_fit_data(
    pointings,
    binning_func,
    energy_range
):
    ebs = [] # do i need this?
    counts = []
    dets = []
    t_elapsed = []
    t_start = []
        
    for pair in pointings:
        time_start1, time_elapsed1, energy_bins1, counts_f1 = extract_pointing_info(pair[0], pair[1])
        time_start2, time_elapsed2, energy_bins2, counts_f2 = extract_pointing_info(pair[0], pair[2])
        t_start.append((time_start1, time_start2))
        
        dets1 = get_live_dets(time=time_start1, event_types=["single"])
        dets2 = get_live_dets(time=time_start2, event_types=["single"])
        assert np.array_equal(dets1, dets2), f"Active detectors are not the same for {pair[1]} and {pair[2]}"
        dets.append(dets1)
        
        t_elapsed.append((time_elapsed1[dets1], time_elapsed2[dets1]))
        
        assert np.array_equal(energy_bins1, energy_bins2), f"Energy bins are not the same for {pair[1]} and {pair[2]}"
        
        # remove this later
        if energy_range[1]:
            for i, e in enumerate(energy_bins1):
                if e > energy_range[1]:
                    energy_bins1 = energy_bins1[:i]
                    counts_f1 = counts_f1[:i]
                    counts_f2 = counts_f2[:i]
                    break
        
        eb, c = binning_func(
            energy_bins1,
            np.append(counts_f1[dets1], counts_f2[dets1], axis=0),
            5,
            120)
        counts.append((c[:len(dets1)], c[len(dets1):]))
        ebs.append(eb)
            
    ebs = tuple(ebs) 
    counts = tuple(counts)
    dets = tuple(dets)
    t_elapsed = tuple(t_elapsed)
    t_start = tuple(t_start)
    
    return ebs, counts, dets, t_elapsed, t_start

In [6]:
def generate_resp_mats(
    pointings,
    ebs,
    emod,
    t_start,
    dets,
    sources,
):
    resp_mats = []
    
    for count, pair in enumerate(pointings):
        version1 = find_response_version(t_start[count][0])
        version2 = find_response_version(t_start[count][1])
        assert version1 == version2, f"Response versions are not equal for {pair[1]} and {pair[2]}"
        
        rsp_base = ResponseDataRMF.from_version(version1)
        source_resp_mats = []
        for source in sources:
            sds1 = np.array([])
            sds2 = np.array([])
            for d in dets[count]:
                rsp1 = ResponseRMFGenerator.from_time(t_start[count][0], d, ebs[count], emod, rsp_base)
                sd1 = SPIDRM(rsp1, source[1].position.get_ra(), source[1].position.get_dec())
                sds1 = np.append(sds1, sd1.matrix.T)
                rsp2 = ResponseRMFGenerator.from_time(t_start[count][1], d, ebs[count], emod, rsp_base)
                sd2 = SPIDRM(rsp2, source[1].position.get_ra(), source[1].position.get_dec())
                sds2 = np.append(sds2, sd2.matrix.T)
            source_resp_mats.append((
                sds1.reshape((len(dets[count]), len(emod)-1, len(ebs[count])-1)),
                sds2.reshape((len(dets[count]), len(emod)-1, len(ebs[count])-1)),
            ))
        resp_mats.append(tuple(source_resp_mats))
        
    resp_mats = tuple(resp_mats)
    
    return resp_mats

In [7]:
def run_multinest_powerlaws( # allow other source types
    pointings,
    emod,
    counts,
    t_elapsed,
    dets,
    resp_mats,
    sources,
):
    num_sources = len(sources)
    
    @njit
    def bmaxba(m1, m2, t1, t2, C1, C2):
        first = C1+C2-(m1+m2)*(t1+t2)
        root = (C1+C2+(m1-m2)*(t1+t2))**2-4*C1*(m1-m2)*(t1+t2)
        res = (first+np.sqrt(root))/(2*(t1+t2))
        if res < 0:
            return 0
        return res
    
    @njit
    def logLcore(spec_binned):
        logL=0
        for k in range(len(pointings)):
            for j in range(len(dets[k])):
                m1 = np.zeros(len(resp_mats[k][0][0][0,0,:]))
                m2 = m1.copy()
                for i in range(num_sources):
                    m1 += np.dot(spec_binned[i,:], resp_mats[k][i][0][j])
                    m2 += np.dot(spec_binned[i,:], resp_mats[k][i][1][j])
                for i in range(len(m1)):
                    bm = bmaxba(m1[i], m2[i], t_elapsed[k][0][j], t_elapsed[k][1][j], counts[k][0][j, i], counts[k][1][j, i])
                    logL += (counts[k][0][j, i]*math.log(t_elapsed[k][0][j]*(m1[i]+bm))+
                            counts[k][1][j, i]*math.log(t_elapsed[k][1][j]*(m2[i]+bm))-
                            t_elapsed[k][0][j]*(m1[i]+bm)-
                            t_elapsed[k][1][j]*(m2[i]+bm))
        return logL

    def logLba_mult(trial_values, ndim=None, params=None):
        spec_binned = np.zeros((num_sources, len(emod)-1))
        for i in range(num_sources):
            sources[i][0].index = trial_values[1 + 2*i]
            sources[i][0].K = trial_values[0 + 2*i]
            spec = sources[i][0](emod)
            spec_binned[i,:] = (emod[1:]-emod[:-1])*(spec[:-1]+spec[1:])/2
        return logLcore(spec_binned)
    
    def prior(params, ndim=None, nparams=None):
        for j, source in enumerate(sources):
            for i, (parameter_name, parameter) in enumerate(
                source[1].free_parameters.items()
            ):
                try:
                    params[i + j*2] = parameter.prior.from_unit_cube(params[i + j*2])

                except AttributeError:
                    raise RuntimeError(
                        "The prior you are trying to use for parameter %s is "
                        "not compatible with sampling from a unitcube"
                        % parameter_name
                    )

    num_params = num_sources * 2
    
    # ###
    # trial_values = [4.5e-3, -2.08, 7e-3, -2.5]
    # spec_binned = np.zeros((len(sources), len(pointings), len(emod)-1))
    # for i in range(len(sources)):
    #     sources[i][0].index = trial_values[1 + 2*i]
    #     sources[i][0].K = trial_values[0 + 2*i]
    #     for j in range(len(pointings)):
    #         spec = sources[i][0](emod)
    #         spec_binned[i,j,:] = (emod[1:]-emod[:-1])*(spec[:-1]+spec[1:])/2
            
    # print(logLcore(spec_binned))
    # return None
    # ###

    if not os.path.exists("./chains"):
        os.mkdir("chains")
    sampler = pymultinest.run(
        logLba_mult, prior, num_params, num_params, n_live_points=800, resume=False, verbose=True
    )

In [8]:
def calc_fit_quality_crab(summary, cov, true_vals):
    fit_val = np.array([summary["Crab K"][1], summary["Crab index"][1]])
    fit_cov = cov[1]
    rel_distance = []
    
    for i in range(len(true_vals)):
        dif = fit_val - true_vals[i]
        
        rel_distance.append(np.sqrt(
            np.linalg.multi_dot([dif, np.linalg.inv(fit_cov), dif])
        ))
        
    return np.array(rel_distance)

In [9]:
def ppc(
    pointings,
    parameters,
    resp_mats,
    ebs,
    counts,
    emod,
    dets,
    t_elapsed,
    sources
):
    for i, pair in enumerate(pointings):
        s1, s2, b = calc_rates_powerlaw(
            parameters[i],
            resp_mats[i],
            ebs[i],
            counts[i],
            emod,
            dets[i],
            t_elapsed[i],
            sources
        )
        

def calc_rates_powerlaw(
    parameters,
    resp_mats,
    eb,
    counts,
    emod,
    dets,
    t_elapsed,
    sources
):
    source_rate1 = np.zeros((len(dets), len(eb)-1), len(parameters))
    source_rate2 = np.zeros((len(dets), len(eb)-1), len(parameters))
    background_rate = np.zeros((len(dets), len(eb)-1), len(parameters))
    
    num_sources = len(sources)
    
    @njit
    def bmaxba(m1, m2, t1, t2, C1, C2):
        first = C1+C2-(m1+m2)*(t1+t2)
        root = (C1+C2+(m1-m2)*(t1+t2))**2-4*C1*(m1-m2)*(t1+t2)
        res = (first+np.sqrt(root))/(2*(t1+t2))
        if res < 0:
            return 0
        return res
    
    for k, params in enumerate(parameters):
        spec_binned = np.zeros((num_sources, len(emod)-1))
        for i in range(num_sources):
            sources[i][0].index = params[1 + 2*i]
            sources[i][0].K = params[0 + 2*i]
            spec = sources[i][0](emod)
            spec_binned[i,:] = (emod[1:]-emod[:-1])*(spec[:-1]+spec[1:])/2
            
        for j in range(len(dets)):
            for i in range(num_sources):
                source_rate1[j,:,k] += np.dot(spec_binned[i,:], resp_mats[i][0][j])
                source_rate2[j,:,k] += np.dot(spec_binned[i,:], resp_mats[i][1][j])
            for i in range(len(eb)-1):
                background_rate[j,i,k] = bmaxba(
                    source_rate1[j,i,k],
                    source_rate2[j,i,k],
                    t_elapsed[0][j],
                    t_elapsed[1][j],
                    counts[0][j, i],
                    counts[1][j, i]
                )

    return source_rate1, source_rate2, background_rate

In [10]:
def multinest_for_pointing_pairs(
    folder,
    pointings,
    sources,
    parameter_names,
    true_values_main,
    binning_func,
    energy_range,
    emod,
):
    
    ebs, counts, dets, t_elapsed, t_start = prepare_fit_data(pointings, binning_func, energy_range)
    
    resp_mats = generate_resp_mats(pointings, ebs, emod, t_start, dets, sources)
    
    run_multinest_powerlaws(pointings, emod, counts, t_elapsed, dets, resp_mats, sources)
        
    parameter_names.extend(["$z$"])
        
    c = ChainConsumer()
    chain = np.loadtxt('./chains/1-post_equal_weights.dat')
    c.add_chain(chain, parameters=parameter_names, name='fit')
    
    fig = c.plotter.plot(filename="crab_parameter_fit.pdf", 
                        parameters=parameter_names[:-1],
                        truth={'Crab K':true_values_main[0,0], 'Crab index':true_values_main[0,1]},
                        figsize="page")
    
    plt.savefig(f"{folder}/parameter_fit_distributions.pdf")
    
    summary = c.analysis.get_summary(parameters=['Crab K', 'Crab index'])
    cov = c.analysis.get_covariance(parameters=['Crab K', 'Crab index'])
    
    rel_distances = calc_fit_quality_crab(summary, cov, true_values_main)
    print(rel_distances)
    
    with open(f"{folder}/reference_values", "w") as f:
        f.write("Crab K Crab index Rel. Dist.\n")
        for i in range(len(true_values_main)):
            f.write(f"{true_values_main[i,0]:.5f} {true_values_main[i,1]:.2f} {rel_distances[i]:.2f}\n")
            

    

In [11]:
def rebin_data_exp(
    bins,
    counts,
    energy_range
):

    if energy_range[0]:
        for i, e in enumerate(bins):
            if e > energy_range[0]:
                bins = bins[i:]
                counts = counts[i:]
                break
    if energy_range[1]:
        for i, e in enumerate(bins):
            if e > energy_range[1]:
                bins = bins[:i]
                counts = counts[:i]
                break
    
    min_counts = 5
    
    max_num_bins = 120
    min_num_bins = 1
    
    finished = False
    
    while not finished:
        num_bins = round((max_num_bins + min_num_bins) / 2)
        
        if num_bins == max_num_bins or num_bins == min_num_bins:
            num_bins = min_num_bins
            finished = True
        
        temp_bins = np.geomspace(bins[0], bins[-1], num_bins+1)
        
        new_bins, new_counts = rebin_closest(bins, counts, temp_bins)
        
        if np.amin(new_counts) < min_counts:
            max_num_bins = num_bins
        else:
            min_num_bins = num_bins
            
    return new_bins, new_counts
    
    
def rebin_closest(bins, counts, temp_bins):
    bin = len(bins) - 1
    for i, e in reversed(list(enumerate(temp_bins))):
        for j in range(bin, -1, -1):
            pass
            
            
counts = np.linspace(1,40,40).reshape((2,20))
bins = np.linspace(1,21,21)
temp_bins = np.geomspace(1,21,11)
rebin_closest(bins, counts, temp_bins)

In [12]:
def rebin_data_full(energy_bins, counts, min_counts, max_num_bins = None):
    max_num_bins = max_num_bins if max_num_bins else len(energy_bins)
    min_num_bins = 1
    
    finished = False
    
    while not finished:
        num_bins = round((max_num_bins + min_num_bins) / 2)
        
        if num_bins == max_num_bins or num_bins == min_num_bins:
            num_bins = min_num_bins
            finished = True
        
        new_bins = np.geomspace(energy_bins[0], energy_bins[-1], num_bins+1)
        
        new_counts = rebin_data(energy_bins, counts, new_bins)
        
        if np.amin(new_counts) < min_counts:
            max_num_bins = num_bins
        else:
            min_num_bins = num_bins
            
    return new_bins, new_counts
    
@njit
def rebin_data(energy_bins, counts, new_energy_bins):
    new_counts = np.zeros((counts.shape[0], len(new_energy_bins)-1))
    
    def enter_bins_smaller(i, bin):
        split = ((energy_bins[bin+1] - max(energy_bins[bin], new_energy_bins[i]))
                 / (energy_bins[bin+1] - energy_bins[bin]))
        new_counts[ : , i] += counts[ : , bin] * split
    
    bin = 0
    length = len(energy_bins) - 1
    for i in range(len(new_energy_bins)-1):
        
        while new_energy_bins[i+1] > energy_bins[bin+1]:
            enter_bins_smaller(i, bin)
            bin += 1
        
        if new_energy_bins[i+1] == energy_bins[bin+1]:
            enter_bins_smaller(i, bin)
            bin += 1
        
        else:
            split = ((min(energy_bins[bin+1], new_energy_bins[i+1]) - max(energy_bins[bin], new_energy_bins[i]))
                     / (energy_bins[bin+1] - energy_bins[bin]))
            new_counts[ : , i] += counts[ : , bin] * split
            
    return new_counts

In [13]:
def crab_sources():
    sources = []
    parameter_names = []
    
    piv = 40
    ra_crab, dec_crab = 83.6333, 22.0144
    crab_parameters = np.array([[9.3, 1, -2.08],
                                [7.52e-4, 100, -1.99],
                                [11.03, 1, -2.1]])
    
    true_values_main = np.zeros((len(crab_parameters),2))
    true_values_main[:,0] = crab_parameters[:,0] * (piv / crab_parameters[:,1])**crab_parameters[:,2]
    true_values_main[:,1] = crab_parameters[:,2]

    crab_pl = Powerlaw()
    crab_pl.index = true_values_main[0,1]
    crab_pl.K = true_values_main[0,0]
    crab_pl.piv = piv
    crab_pl.K.prior = Log_uniform_prior(lower_bound=1e-6, upper_bound=1e-0)
    crab_pl.index.prior = Uniform_prior(lower_bound=-4, upper_bound=0)
    crab_component1 = SpectralComponent('crab_pl',shape=crab_pl)
    crab_ps = PointSource('crab_plline',ra=ra_crab, dec=dec_crab, components=[crab_component1])
    
    sources.append((crab_pl, crab_ps))
    parameter_names.extend(["Crab K", "Crab index"])
    
    ra_1A, dec_1A = 84.7270, 26.3160
    _1A_pl = Powerlaw()
    # _1A_pl.index = -2
    # _1A_pl.K = 0.005
    _1A_pl.piv = piv
    _1A_pl.K.prior = Log_uniform_prior(lower_bound=1e-7, upper_bound=1e1)
    _1A_pl.index.prior = Uniform_prior(lower_bound=-10, upper_bound=4)
    _1A_component1 = SpectralComponent('_1A_pl',shape=_1A_pl)
    _1A_ps = PointSource('_1A_plline',ra=ra_1A, dec=dec_1A, components=[_1A_component1])
    
    sources.append((_1A_pl, _1A_ps))
    parameter_names.extend(["1A K", "1A index"])
    
    return tuple(sources), parameter_names, true_values_main


In [14]:
def crab_sources_extended():
    sources = []
    parameter_names = []
    
    piv = 40
    ra_crab, dec_crab = 83.6333, 22.0144
    crab_parameters = np.array([[9.3, 1, -2.08],
                                [7.52e-4, 100, -1.99],
                                [11.03, 1, -2.1]])
    
    true_values_main = np.zeros((len(crab_parameters),2))
    true_values_main[:,0] = crab_parameters[:,0] * (piv / crab_parameters[:,1])**crab_parameters[:,2]
    true_values_main[:,1] = crab_parameters[:,2]

    crab_pl = Powerlaw()
    crab_pl.index = true_values_main[0,1]
    crab_pl.K = true_values_main[0,0]
    crab_pl.piv = piv
    crab_pl.K.prior = Log_uniform_prior(lower_bound=1e-6, upper_bound=1e-0)
    crab_pl.index.prior = Uniform_prior(lower_bound=-4, upper_bound=0)
    crab_component1 = SpectralComponent('crab_pl',shape=crab_pl)
    crab_ps = PointSource('crab_plline',ra=ra_crab, dec=dec_crab, components=[crab_component1])
    
    sources.append((crab_pl, crab_ps))
    parameter_names.extend(["Crab K", "Crab index"])
    
    ra_1A, dec_1A = 84.7270, 26.3160
    _1A_pl = Powerlaw()
    # _1A_pl.index = -2
    # _1A_pl.K = 0.005
    _1A_pl.piv = piv
    _1A_pl.K.prior = Log_uniform_prior(lower_bound=1e-7, upper_bound=1e1)
    _1A_pl.index.prior = Uniform_prior(lower_bound=-10, upper_bound=4)
    _1A_component1 = SpectralComponent('_1A_pl',shape=_1A_pl)
    _1A_ps = PointSource('_1A_plline',ra=ra_1A, dec=dec_1A, components=[_1A_component1])
    
    sources.append((_1A_pl, _1A_ps))
    parameter_names.extend(["1A K", "1A index"])
    
    ra_4U05, dec_4U05 = 77.6896, 16.4986
    _4U05_pl = Powerlaw()
    # _4U05_pl.index = -2
    # _4U05_pl.K = 0.005
    _4U05_pl.piv = piv
    _4U05_pl.K.prior = Log_uniform_prior(lower_bound=1e-7, upper_bound=1e1)
    _4U05_pl.index.prior = Uniform_prior(lower_bound=-10, upper_bound=4)
    _4U05_component1 = SpectralComponent('_4U05_pl',shape=_4U05_pl)
    _4U05_ps = PointSource('_4U05_plline',ra=ra_4U05, dec=dec_4U05, components=[_4U05_component1])
    
    sources.append((_4U05_pl, _4U05_ps))
    parameter_names.extend(["4U05 K", "4U05 index"])
    
    ra_4U06, dec_4U06 = 94.2800, 9.13700
    _4U06_pl = Powerlaw()
    # _4U06_pl.index = -2
    # _4U06_pl.K = 0.005
    _4U06_pl.piv = piv
    _4U06_pl.K.prior = Log_uniform_prior(lower_bound=1e-7, upper_bound=1e1)
    _4U06_pl.index.prior = Uniform_prior(lower_bound=-10, upper_bound=4)
    _4U06_component1 = SpectralComponent('_4U06_pl',shape=_4U06_pl)
    _4U06_ps = PointSource('_4U06_plline',ra=ra_4U06, dec=dec_4U06, components=[_4U06_component1])
    
    sources.append((_4U06_pl, _4U06_ps))
    parameter_names.extend(["4U06 K", "4U06 index"])
    
    ra_geminga, dec_geminga = 98.4750, 17.7670
    geminga_pl = Powerlaw()
    # geminga_pl.index = -2
    # geminga_pl.K = 0.005
    geminga_pl.piv = piv
    geminga_pl.K.prior = Log_uniform_prior(lower_bound=1e-7, upper_bound=1e1)
    geminga_pl.index.prior = Uniform_prior(lower_bound=-10, upper_bound=4)
    geminga_component1 = SpectralComponent('geminga_pl',shape=geminga_pl)
    geminga_ps = PointSource('geminga_plline',ra=ra_geminga, dec=dec_geminga, components=[geminga_component1])
    
    sources.append((geminga_pl, geminga_ps))
    parameter_names.extend(["geminga K", "geminga index"])
    
    return tuple(sources), parameter_names, true_values_main

In [15]:
folder = "orbit_1019"

# save_pair_clusters(
#     create_pair_clusters_crab(
#         ("crab_data/1019",),
#         min_angle_dif=1.5,
#         max_angle_dif=4.,
#         max_time_dif=0.2,
#         radius_around_crab=5.,
#         min_time_elapsed=600.
#     ),
#     folder
# )

pointings = load_pair_clusters(folder)
sources, parameter_names, true_values_main = crab_sources()

multinest_for_pointing_pairs(
    folder,
    pointings,
    sources,
    parameter_names,
    true_values_main,
    rebin_data_full,
    (None, 80),
    np.geomspace(18, 150, 50)
)

Using the irfs that are valid between 10/05/27 12:45:00 and present (YY/MM/DD HH:MM:SS)
Using the irfs that are valid between 10/05/27 12:45:00 and present (YY/MM/DD HH:MM:SS)
Using the irfs that are valid between 10/05/27 12:45:00 and present (YY/MM/DD HH:MM:SS)
Using the irfs that are valid between 10/05/27 12:45:00 and present (YY/MM/DD HH:MM:SS)
Using the irfs that are valid between 10/05/27 12:45:00 and present (YY/MM/DD HH:MM:SS)
Using the irfs that are valid between 10/05/27 12:45:00 and present (YY/MM/DD HH:MM:SS)
Using the irfs that are valid between 10/05/27 12:45:00 and present (YY/MM/DD HH:MM:SS)
Using the irfs that are valid between 10/05/27 12:45:00 and present (YY/MM/DD HH:MM:SS)
Using the irfs that are valid between 10/05/27 12:45:00 and present (YY/MM/DD HH:MM:SS)
Using the irfs that are valid between 10/05/27 12:45:00 and present (YY/MM/DD HH:MM:SS)
Using the irfs that are valid between 10/05/27 12:45:00 and present (YY/MM/DD HH:MM:SS)
Using the irfs that are valid be

In [89]:
folder = "orbit_1019_extended"

# save_pair_clusters(
#     create_pair_clusters_crab(
#         ("crab_data/1019",),
#         min_angle_dif=1.5,
#         max_angle_dif=4.,
#         max_time_dif=0.2,
#         radius_around_crab=5.,
#         min_time_elapsed=600.
#     ),
#     folder
# )

pointings = load_pair_clusters(folder)
sources, parameter_names, true_values_main = crab_sources_extended()

multinest_for_pointing_pairs(
    folder,
    pointings,
    sources,
    parameter_names,
    true_values_main,
    rebin_data_full,
    (None, 80),
    np.geomspace(18, 150, 50)
)



[8.34564566 2.40446024 8.11489942]
